In [1]:
import pandas as pd
import os
from pycaret.classification import *

In [2]:
data_loc='./data'

In [3]:
testing=pd.read_csv(os.path.join(data_loc,'new_test.csv'))
testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   10000 non-null  int64  
 1   duration                     10000 non-null  int64  
 2   protocol_type                10000 non-null  object 
 3   service                      10000 non-null  int64  
 4   flag                         10000 non-null  int64  
 5   src_bytes                    10000 non-null  int64  
 6   dst_bytes                    10000 non-null  int64  
 7   land                         10000 non-null  int64  
 8   wrong_fragment               10000 non-null  int64  
 9   urgent                       10000 non-null  int64  
 10  hot                          10000 non-null  int64  
 11  num_failed_logins            10000 non-null  int64  
 12  logged_in                    10000 non-null  int64  
 13  num_compromised  

Changing the datatypes and its column type to Float and Categorical

In [4]:
# cat_features=['protocol_type','service','flag','xAttack']
# temp_data=testing.drop(columns=cat_features)
# num_features=temp_data.columns.tolist()
# print(num_features)

In [5]:
# updated_data=testing.copy()
# for i in updated_data:
#     if i in num_features:
#         updated_data[i]=updated_data[i].astype(float)
#     else:
#         updated_data[i]=updated_data[i].astype('category')

# updated_data.info()

In [6]:
# droplist=['xAttack','urgent','num_shells', 'root_shell',
#         'num_failed_logins', 'num_access_files','dst_host_srv_diff_host_rate',
#         'diff_srv_rate', 'dst_host_diff_srv_rate', 'wrong_fragment','num_outbound_cmds']
# updated_data.drop(axis=0,columns=droplist,inplace=True)
# updated_data.info()

In [7]:
prediction=load_model('./model/latest_intrusion_model')

Transformation Pipeline and Model Successfully Loaded


In [8]:
prediction_df=predict_model(estimator=prediction,data=testing)
prediction_df.head()

,Unnamed: 0,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,xAttack,Label,Score
0,202,0,icmp,25,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,110,110,1.0,1.0,0.0,0.0,1.0,0.0,0.00,255,255,1.00,0.00,0.00,0.00,0.65,0.65,0.32,0.32,dos,dos,0.5
1,768,0,icmp,25,2,312,1856,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,9,0.0,0.0,0.0,0.0,1.0,0.0,0.33,1,254,1.00,0.00,1.00,0.06,0.00,0.00,0.00,0.00,normal,normal,1.0
2,3753,0,icmp,25,2,245,2058,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.00,114,255,1.00,0.00,0.01,0.05,0.01,0.00,0.00,0.00,normal,normal,1.0
3,20087,0,icmp,25,2,298,1267,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,18,0.0,0.0,0.0,0.0,1.0,0.0,0.11,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,normal,1.0
4,16159,0,icmp,20,2,740,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.00,104,19,0.18,0.03,0.18,0.00,0.00,0.00,0.00,0.00,normal,normal,1.0


In [9]:
prediction_df['Label'].value_counts()

normal    6495
dos       2677
probe      819
r2l          9
Name: Label, dtype: int64

In [10]:
label_info = pd.DataFrame(prediction_df['Label'].value_counts().reset_index().values, columns=["Label", "No. of Data"])
print(label_info.head())
total_no_of_data=label_info['No. of Data'].sum()
print('\nTotal Number of Data: ',total_no_of_data)

    Label No. of Data
0  normal        6495
1     dos        2677
2   probe         819
3     r2l           9

Total Number of Data:  10000


In [11]:
temp=label_info[label_info['Label']=='normal']
normal_count=int(temp['No. of Data'])
anomaly_count=total_no_of_data-normal_count
anomaly_percent=anomaly_count/total_no_of_data*100
if anomaly_count>25:
    print("Moderate")
elif anomaly_count>50:
    print("Severe")
else:
    print("Good")
anomaly_info={
        'Type':['normal','anomaly'],
        'No. of Data':[normal_count,anomaly_count]}
anomaly_info=pd.DataFrame.from_dict(anomaly_info)
print(anomaly_info)

Moderate
      Type  No. of Data
0   normal         6495
1  anomaly         3505


In [12]:
print("Accuracy of the model for the given data = ",round((prediction_df['Score'].mean()*100),4))

Accuracy of the model for the given data =  92.2389


In [13]:
label_items=prediction_df['Label'].unique().tolist()
print(label_items)
accuracy=[]
for i in label_items:
    temp=prediction_df[prediction_df['Label']==i]
    accuracy.append(round(temp['Score'].mean()*100,4))
print(accuracy)
mean_score={
    'Type':label_items,
    'Accuracy Score': accuracy
}
mean_score=pd.DataFrame.from_dict(mean_score)
print(mean_score)

['dos', 'normal', 'probe', 'r2l']
[95.3085, 91.1269, 91.4432, 54.1111]
     Type  Accuracy Score
0     dos         95.3085
1  normal         91.1269
2   probe         91.4432
3     r2l         54.1111


In [14]:
import chart_studio
import chart_studio.tools as tls
import chart_studio.plotly as py
import plotly.express as px


In [15]:
plot1=px.pie(label_info,names=label_info['Label'],values=label_info['No. of Data'],title=('No. of Data in each Category\nTotal Data: {0}'.format(total_no_of_data)))
plot1

In [16]:
plot1_push=py.plot(plot1,filename='Intrusion Detection - No. of Data in each Category',auto_open=False)
fig1=tls.get_embed(plot1_push)
print(fig1)

<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plotly.com/~VijayasaiVS/96.embed" height="525" width="100%"></iframe>
